# Training Model with V2 Datas


### Initializing GPU

In [15]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


### Imports

In [16]:

from tensorflow import keras
import numpy as np
import json
from sklearn.model_selection import train_test_split
import os
import  random
import numpy as np
from tensorflow.keras import layers
import keras as keras
from keras.applications import  Xception
from PIL import Image
import random
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(4)
np.set_printoptions(precision=3, suppress=True)




2.19.0
Num GPUs Available:  1


### Global variables

In [27]:

IS_WSL = True
WINDOWS_DATA_FOLDER="/mnt/e/code/ai/data/tcg/"
GENERATED_DATAS_V2_FOLDER_PATH = "data/generated_datas_v1/"
TCG_FOLDER = "data/tcg/"

if IS_WSL :
    GENERATED_DATAS_V2_FOLDER_PATH = WINDOWS_DATA_FOLDER + GENERATED_DATAS_V2_FOLDER_PATH
    TCG_FOLDER = WINDOWS_DATA_FOLDER +  TCG_FOLDER
JSONS_FOLDER=GENERATED_DATAS_V2_FOLDER_PATH+"json/"
IMAGES_FOLDER=GENERATED_DATAS_V2_FOLDER_PATH+"images/"

RANDOM_STATE=42
RESIZE_SIZE=224
RGB = True
EPOCHS = 10
BATCH_SIZE = 28
TEST_SIZE = 0.25
MIN_ASSETS=0
MAX_ASSETS=35000
if RGB:
    RGB_VALUE = 3
else:
    RGB_VALUE = 1



In [18]:

def load_files_list():

    json_files_list = []
    for json_file in os.listdir(JSONS_FOLDER):
        if json_file.endswith(".json"):
            json_files_list.append(json_file)
           

    random.Random(RANDOM_STATE).shuffle(json_files_list)
    json_files_list = json_files_list[MIN_ASSETS:MAX_ASSETS]
    print("Total files : ",json_files_list.__len__())
    print("First file : ",json_files_list[0])
    return json_files_list



### Loading files in generator

In [ ]:
def read_json_from_s3(json_key):
    try:
        with open(JSONS_FOLDER + json_key) as json_data:
            raw_json = json.load(json_data)
            image_key = IMAGES_FOLDER + raw_json["asset"]["name"]
            points = raw_json["regions"][0]["points"]
            with open( TCG_FOLDER+ raw_json["card_id"]+ ".json") as json_data_full:
                raw_json_full = json.load(json_data_full)
                x0 = points[0]["x"]
                y0 = points[0]["y"] 
                x1 = points[1]["x"] 
                y1 = points[1]["y"] 
                x2 = points[2]["x"]
                y2 = points[2]["y"]
                x3 = points[3]["x"]
                y3 = points[3]["y"]
                new_json = {
                    "image_key": image_key,
                    "bounding_boxes": [x0, y0, x1, y1, x2, y2, x3, y3],
                    "name" : raw_json_full["name"],
                    "id" : raw_json_full["id"],
                    "set_id" : raw_json_full["set"]["id"],
                    "number" : raw_json_full["number"],
                    "rarity" : raw_json_full["rarity"],
                    "artist" : raw_json_full["artist"],
                }
                return new_json
            return None
        return None
    except :
        return None
# Fonction pour lire les images depuis S3
def read_image_from_s3(image_key):
    try:
        img = Image.open(image_key)
        img = img.resize((RESIZE_SIZE, RESIZE_SIZE))
        img = np.asarray( img, dtype="int32" )
        np.array(img)
        img = img/255
        return img
    except Exception as e:
        print(e)
        return None
# Fonction pour créer un dataset TensorFlow
def create_dataset(json_keys, batch_size=32):
    def generator():
        for json_key in json_keys:
            annotations = read_json_from_s3(json_key)
            if annotations == None or annotations["image_key"] == None :
                print("File : " + json_key + " is Malformated, delete it")
                continue
            img = read_image_from_s3(annotations["image_key"])
            if img is None:
                print("Image : " + annotations["image_key"] + " is Malformated, delete it")
                continue
            yield img, {"bb_head": annotations['bounding_boxes'],
                        "name" : annotations['name'],
                        "id" : annotations['id'],
                        "set_id" : annotations['set_id'],
                        "number" : annotations['number'],
                        "rarity" : annotations['rarity'],
                        "artist" : annotations['artist'],
                        }
    dataset = tf.data.Dataset.from_generator(generator,
                                             output_signature=(
                                                 tf.TensorSpec(shape=(RESIZE_SIZE, RESIZE_SIZE, RGB_VALUE), dtype=tf.float32),
                                                    {
                                                     "bb_head": tf.TensorSpec(shape=(8,), dtype=tf.float32),
                                                     "name" : tf.TensorSpec(shape=(), dtype=tf.string),
                                                     "id" : tf.TensorSpec(shape=(), dtype=tf.string),
                                                     "set_id" : tf.TensorSpec(shape=(), dtype=tf.string),
                                                     "number" : tf.TensorSpec(shape=(), dtype=tf.string),
                                                     "rarity" : tf.TensorSpec(shape=(), dtype=tf.string),
                                                     "artist" : tf.TensorSpec(shape=(), dtype=tf.string),
                                                     
                                                 }
                                             ))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

### Load datasets

In [29]:
json_files_list = load_files_list()

train_files, val_files = train_test_split(json_files_list, shuffle=True, random_state=42, test_size=TEST_SIZE)
print("Train files : ", len(train_files))
print("Val files : ", len(val_files))
train_dataset = create_dataset(train_files, BATCH_SIZE)
val_dataset = create_dataset(val_files, BATCH_SIZE)


Total files :  35000
First file :  gym1-108_-3_1_-6_584_39.json
Train files :  26250
Val files :  8750


In [ ]:
# Ajouter une couche dense pour les embeddings
def normalize_embedding(x):
    return tf.math.l2_normalize(x, axis=1)

# Définir explicitement la couche d'entrée
input_layer = layers.Input(shape=(RESIZE_SIZE, RESIZE_SIZE, RGB_VALUE))
xception = Xception(weights="imagenet", include_top=False)(input_layer)


# Branche pour l'embedding
embedding = layers.GlobalAveragePooling2D()(xception)
embedding = layers.Dense(256, activation=None, name='embedding')(embedding)
embedding = layers.Lambda(normalize_embedding, name='normalized_embedding')(embedding)

# Construire le reste du modèle
flatten = layers.Flatten(name='flatten')(xception)
locator_branch = layers.Dense(128, activation='relu', name='bb_1')(flatten)
locator_branch = layers.Dense(64, activation='relu', name='bb_2')(locator_branch)
locator_branch = layers.Dense(32, activation='relu', name='bb_3')(locator_branch)
locator_branch = layers.Dense(8, activation='sigmoid', name='bb_head')(locator_branch)

# Créer le modèle multi-tâches
model = tf.keras.Model(
    inputs=input_layer,
    outputs={
        "bb_head": locator_branch,
        "embedding": embedding
    }
)
# Définir les pertes et métriques
losses = {
    "bb_head": tf.keras.losses.MeanSquaredError()
}

metrics = {
    "bb_head": ["accuracy"]
}

# Compiler le modèle
model.compile(
    optimizer='Adam',
    loss=losses,
    metrics=metrics
)


In [31]:

# Callbacks
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='../checkpoints/model_checkpoint_{epoch:02d}.keras',
    save_best_only=True,
    monitor='val_bb_head_accuracy',
    mode='max',
    save_weights_only=False,
    verbose=1
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_bb_head_accuracy',
    patience=3,
    mode='max',
    verbose=1
)

In [32]:

# Entraînement du modèle
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=12,
    callbacks=[checkpoint,early_stopping]
)


Epoch 1/12
     78/Unknown 454s 5s/step - bb_head_accuracy: 0.2672 - loss: 0.0805

2025-11-06 17:26:42.833286: W tensorflow/core/framework/op_kernel.cc:1844] UNKNOWN: FileNotFoundError: [Errno 2] No such file or directory: '/mnt/e/code/ai/data/tcg/data/tcg/cel25c-113.json'
Traceback (most recent call last):

  File "/home/guillianv/miniconda3/envs/tf/lib/python3.12/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/home/guillianv/miniconda3/envs/tf/lib/python3.12/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/home/guillianv/miniconda3/envs/tf/lib/python3.12/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/tmp/ipykernel_1422/1811985888.py", line 40, in generator
    annotations = read_json_from_s3(json_key)
      

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) UNKNOWN:  FileNotFoundError: [Errno 2] No such file or directory: '/mnt/e/code/ai/data/tcg/data/tcg/cel25c-113.json'
Traceback (most recent call last):

  File "/home/guillianv/miniconda3/envs/tf/lib/python3.12/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/home/guillianv/miniconda3/envs/tf/lib/python3.12/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/home/guillianv/miniconda3/envs/tf/lib/python3.12/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             

In [ ]:
model.save("model.keras")